In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/gender_submission.csv")

In [ ]:

train_df

In [ ]:

test_df

In [ ]:

submission

In [ ]:

import random

np.random.seed(1234)
random.seed(1234)

In [ ]:

train_df.shape

In [ ]:

test_df.shape

In [ ]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)
print(train_df)

In [ ]:

train_df.head(100)

In [ ]:
train_df.dtypes

In [ ]:
train_df.describe()

In [ ]:
train_df["Sex"].value_counts()

In [ ]:
train_df["Embarked"][train_df["Embarked"] == "S"][train_df["Survived"] == 1].count()

In [ ]:
train_df["Embarked"][train_df["Embarked"] == "S"][train_df["Survived"] == 0].count()

In [ ]:
train_df["Cabin"].value_counts().head(50)

In [ ]:

train_df.isnull().sum()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
plt.style.use("ggplot")

In [ ]:
train_df[["Embarked", "Survived", "PassengerId"]]

In [ ]:
train_df[["Embarked", "Survived", "PassengerId"]].dropna()

In [ ]:
train_df[["Embarked", "Survived", "PassengerId"]].dropna().groupby(
    ["Embarked", "Survived"]
).count()

In [ ]:
embarked_df = (
    train_df[["Embarked", "Survived", "PassengerId"]]
    .dropna()
    .groupby(["Embarked", "Survived"])
    .count()
    .unstack()
)

In [ ]:

embarked_df.plot.bar(stacked=True)

In [ ]:
embarked_df["survived_rate"] = embarked_df.iloc[:, 1] / (
    embarked_df.iloc[:, 0] + embarked_df.iloc[:, 1]
)

In [ ]:
embarked_df

In [ ]:
sex_df = (
    train_df[["Sex", "Survived", "PassengerId"]]
    .dropna()
    .groupby(["Sex", "Survived"])
    .count()
    .unstack()
)

In [ ]:
sex_df

In [ ]:
sex_df.plot.bar(stacked=True)

In [ ]:
sex_df["Survived_rate"] = sex_df.iloc[:, 1] / (sex_df.iloc[:, 0] + sex_df.iloc[:, 1])

In [ ]:
sex_df

In [ ]:
ticket_df = (
    train_df[["Pclass", "Survived", "PassengerId"]]
    .dropna()
    .groupby(["Pclass", "Survived"])
    .count()
    .unstack()
)

In [ ]:
ticket_df.plot.bar(stacked=True)

In [ ]:
ticket_df["Survived_rate"] = ticket_df.iloc[:, 1] / (
    ticket_df.iloc[:, 0] + ticket_df.iloc[:, 1]
)

In [ ]:
ticket_df

In [ ]:
plt.hist(
    x=[train_df.Age[train_df.Survived == 0], train_df.Age[train_df.Survived == 1]],
    bins=8,
    histtype="barstacked",
    label=["Dead", "Survived"],
)
plt.legend()

In [ ]:
train_df_corr = pd.get_dummies(train_df, columns=["Sex"], drop_first=True)
train_df_corr = pd.get_dummies(train_df_corr, columns=["Embarked"])

In [ ]:
train_df_corr.head()

In [ ]:
train_corr = train_df_corr.corr()

In [ ]:
train_corr

In [ ]:
plt.figure(figsize=(9, 9))
sns.heatmap(train_corr, vmax=1, vmin=-1, center=0, annot=True)

In [ ]:
all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)

In [ ]:
all_df

In [ ]:
all_df.insnull().sum()

In [ ]:
Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()

In [ ]:
Fare_mean.columns = ["Pclass", "Fare"]

In [ ]:
Fare_mean

In [ ]:
all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
all_df.loc[(all["Fare"].isunll()), "Fare"] = all_df["Fare_mean"]
all_df = all_df.drop("Fare_mean", axis=1)

In [ ]:
all_df["Name"].head(5)

In [ ]:
name_df = all_df["Name"].str.split("[,.]", 2, expand=True)

In [ ]:
name_df.columns = ["family_name", "name"]

In [ ]:
name_df

In [ ]:
name_df["family_name"] = name_df["family_name"].str.strip()
name_df["honorific"] = name_df["honorific"].str.strip()
name_df["name"] = name_df["name"].str.strip()

In [ ]:
name_df["honorific"].value_counts()

In [ ]:
all_df = pd.concat([all_df, name_df], axis=1)

In [ ]:
all_df

In [ ]:
plt.figure(figsize=(18, 5))
sns.boxplot(x="honorific", y="Age", data=all_df)

In [ ]:
all_df[["Age", "honorific"]].groupby("honorific").mean()

In [ ]:
train_df = pd.concat([train_df, name_df[0:len(train_df)].reset_index(drop=True)], axis=1)
test_df = pd.concat([train_df, name_df[len(train_df):].reset_index(drop=True)], axis=1)

In [ ]:
honorific_df = train_df[["honorific", "Survived", "PassengerId"]].dropna().groupby(["honorific", "Survived"]).count().unstack()
honorific_df.plot.bar(stacked=True)

In [ ]:
honorific_age_mean = all_df[["honorific","Age"]].groupby("honorific").mean().reset_index()
honorific_age_mean.columns = ["honorific","honorific_Age"]
all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_Age"]
all_df = all_df.drop(["honorific_Age"], axis=1)

In [ ]:
all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
all_df["family_num" ].value_counts()

In [ ]:
all_df.loc[all_df["family_num"]==0, "alone"] = 1
all_df[ "alone" ].fillna(0, inplace=True)

In [ ]:
alldf = all_df.drop(["passengerId","Name", "family_name", "name", "Ticket", "Cabin"]
axis=1)

In [ ]:
all_df.head()

In [ ]:
categories = all_df.columns[all_df.dtypes=="object"]
print(categories)

In [ ]:
all_df.loc[~((all_df["honorific"]=="Mr") | (all_df["honorific"]=="Miss") | (all_df["honorific"]=="Mrs") | (all_df["honorific"]=="Master")),"honorififc"] = "other"

In [ ]:
all_df.honorific.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
all_df["Embarked"].fillna("missing", inplace=True)

In [ ]:
all_df.head()

In [ ]:
le = LabelEncoder()
le = le.fit(all_df["Sex"])
all_df["Sex"] = le.transform(all_df["Sex"])

In [ ]:
for cat in categories:
    le = LabelEncoder()
    print(cat)
    if all_df[cat].dtypes == "object":
        le = le.fit(all_df[cat])
        all_df[cat] = le.transform(all_df[cat])

In [ ]:
all_df.head()

In [ ]:
train_X = all_df[~all_df["Suvived"].install()].drop("Survived", axis=1).reset_index(drop=True)
train_Y = train_df["Survived"]
test_X = all_df[~all_df["Suvived"].install()].drop("Survived", axis=1).reset_index(drop=True)

In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_Y, test_size=0.2)